<a href="https://colab.research.google.com/github/BhavaniPrasadBhavani/Deepfake-Image-detection-2.0/blob/main/ramboSquare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from transformers import pipeline
import cv2
import numpy as np
from PIL import Image
import gradio as gr
import warnings
warnings.filterwarnings('ignore')

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define CNN Model
class DeepfakeDetector(nn.Module):
    def __init__(self):
        super(DeepfakeDetector, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = self.pool(self.relu(self.conv4(x)))
        x = x.view(-1, 256 * 14 * 14)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))
        return x

model = DeepfakeDetector().to(device)
print("Model initialized successfully")

# AI Detection System
class AIDetectionSystem:
    def __init__(self):
        self.model_loaded = False
        self.detector = None
        self.load_model()

    def load_model(self):
        try:
            self.detector = pipeline(
                "image-classification",
                model="dima806/deepfake_vs_real_image_detection",
                device=0 if torch.cuda.is_available() else -1
            )
            self.model_loaded = True
            print("AI detection model loaded successfully")
        except Exception as e:
            print(f"Warning: Could not load AI model - {e}")
            self.model_loaded = False

    def analyze_image(self, image):
        if not self.model_loaded:
            return None, 0.0
        try:
            result = self.detector(image)
            if isinstance(result, list) and len(result) > 0:
                label = result[0]['label'].upper()
                confidence = result[0]['score']
                is_fake = 'FAKE' in label
                return is_fake, confidence
            return None, 0.0
        except Exception as e:
            print(f"Analysis failed: {e}")
            return None, 0.0

    def get_detailed_analysis(self, image):
        is_fake, confidence = self.analyze_image(image)
        if is_fake is None:
            return {"status": "error", "message": "Unable to analyze image"}
        if confidence > 0.8:
            certainty = "High"
        elif confidence > 0.6:
            certainty = "Medium"
        else:
            certainty = "Low"
        return {
            "status": "success",
            "prediction": "Synthetic" if is_fake else "Authentic",
            "confidence": confidence,
            "certainty_level": certainty,
            "risk_score": confidence if is_fake else (1 - confidence)
        }

ai_system = AIDetectionSystem()

# Image Processing

def process_image(image_input):
    try:
        if isinstance(image_input, Image.Image):
            image = image_input
        elif isinstance(image_input, np.ndarray):
            if len(image_input.shape) == 3 and image_input.shape[2] == 3:
                image_input = cv2.cvtColor(image_input, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image_input)
        elif isinstance(image_input, str):
            image = Image.open(image_input)
        else:
            raise ValueError("Unsupported image format")
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image = image.resize((224, 224), Image.Resampling.LANCZOS)
        return image
    except Exception as e:
        print(f"Image processing error: {e}")
        return None

def create_tensor(image):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

# Detection Function
def detect_deepfake(image_input, detection_method="AI Model"):
    processed_image = process_image(image_input)
    if processed_image is None:
        return {"error": "Failed to process image"}
    results = {
        "image_processed": True,
        "dimensions": processed_image.size,
        "method": detection_method
    }
    if detection_method == "AI Model":
        analysis = ai_system.get_detailed_analysis(processed_image)
        if analysis["status"] == "success":
            results.update({
                "prediction": analysis["prediction"],
                "confidence": analysis["confidence"],
                "certainty": analysis["certainty_level"],
                "risk_score": analysis["risk_score"]
            })
        else:
            results["error"] = analysis["message"]
    elif detection_method == "Basic CNN":
        tensor_img = create_tensor(processed_image).to(device)
        with torch.no_grad():
            output = model(tensor_img)
            prediction = output.item()
        results.update({
            "prediction": "Synthetic" if prediction > 0.5 else "Authentic",
            "confidence": abs(prediction - 0.5) * 2,
            "certainty": "Demo only",
            "note": "Basic model is for demonstration purposes"
        })
    return results

# Gradio Interface
def create_professional_interface():
    def analyze_uploaded_image(image, method):
        if image is None:
            return "Please upload an image to analyze."
        results = detect_deepfake(image, method)
        if "error" in results:
            return f"Analysis Error: {results['error']}"
        bullet_disclaimer = (
            "\n\nDisclaimer:\n"
            "• This is just for educational purposes only.\n"
            "• Results might not be accurate as the context might be new to the pretrained model.\n"
            "• Some deepfake algorithms might bypass these checks.\n"
            "• Use at your own discretion.\n"
            "• The Basic CNN is not well trained.\n"
            "• Your data is not saved in any server."
        )
        if method == "AI Model":
            return (
                f"Analysis Complete\n\n"
                f"Prediction: {results['prediction']}\n"
                f"Confidence: {results['confidence']:.1%}\n"
                f"Certainty Level: {results['certainty']}\n\n"
                f"Image size: {results['dimensions'][0]}×{results['dimensions'][1]}\n"
                f"Method: {results['method']}"
                f"{bullet_disclaimer}"
            )
        else:
            return (
                f"Analysis Complete\n\n"
                f"Prediction: {results['prediction']}\n"
                f"Confidence: {results['confidence']:.1%}\n\n"
                f"(Basic CNN demo mode)"
                f"{bullet_disclaimer}"
            )

    background_url = "https://raw.githubusercontent.com/BhavaniPrasadBhavani/Deepfake-Image-detection-2.0/main/asf.jpg"
    custom_css = f"""
    body {{
        overflow-y: scroll !important;
    }}
    .gradio-container {{
        background: url('{background_url}') no-repeat center center fixed;
        background-size: cover;
        height: auto !important;
        min-height: 100vh;
    }}
    .gradio-container .gr-box {{
        background-color: rgba(0,0,0,0.6) !important;
        border-radius: 10px;
    }}
    .gradio-container .gr-button {{
        background: linear-gradient(90deg, #4f46e5, #7c3aed) !important;
        color: white !important;
    }}
    .gradio-container .gr-button:hover {{
        transform: translateY(-1px) !important;
    }}
    """

    interface = gr.Interface(
        fn=analyze_uploaded_image,
        inputs=[
            gr.Image(type="pil", label="Upload Image", height=400),
            gr.Radio(["AI Model", "Basic CNN"], value="AI Model", label="Detection Method")
        ],
        outputs=gr.Textbox(label="Analysis Results", lines=12),
        title="Deepfake Detection System",
        description=(
            "Upload an image to detect deepfakes using AI or demo CNN.\n\n"
            "• This is just for educational purposes only.\n"
            "• Results might not be accurate as the context might be new to the pretrained model.\n"
            "• Some deepfake algorithms might bypass these checks.\n"
            "• Use at your own discretion.\n"
            "• The Basic CNN is not well trained.\n"
            "• Your data is not saved in any server."
        ),
        theme=gr.themes.Soft(),
        css=custom_css
    )
    return interface

# App Launch
if __name__ == "__main__":
    interface = create_professional_interface()
    interface.launch(share=True, server_name="0.0.0.0", quiet=False)


Using device: cuda
Model initialized successfully


Device set to use cuda:0


AI detection model loaded successfully
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9ddf40f57bf15378aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
